In [0]:
cd '/content/drive/My Drive/PD/cis-pd'

/content/drive/My Drive/PD/cis-pd


In [0]:
import numpy as np
import pickle
import os
from sklearn.externals import joblib


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [0]:
with open("train_Total_HMM_1.pkl", "rb") as file: 
  train_HMM = pickle.load(file)
with open("cis-pd-y_train.pkl", "rb") as file: 
  labels = pickle.load(file)

In [0]:
train_HMM.shape

(1858, 100)

In [0]:
import pandas as pd
import sklearn.svm as svm
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.model_selection import cross_val_score,ShuffleSplit,StratifiedShuffleSplit

In [0]:
labels_match = {'on_off':0,'dyskinesia':1,'tremor':2}
l = 'tremor'
Y = labels[:,labels_match[l]]

not_nan = ~np.isnan(Y)
X = train_HMM[not_nan]
Y = Y[not_nan]

seed = np.random.randint(100)
trainX,testX,trainY,testY = train_test_split(X,Y,random_state = 20,test_size = 0.25)

print(X.shape)

(1462, 100)


In [0]:
svc = svm.SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=5000, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
svc.fit(trainX,trainY)

pred_trainY = svc.predict(trainX)
pred_testY = svc.predict(testX)
print("SVC_train accuracy rate: {} F1-score: {}".format(svc.score(trainX,trainY),f1_score(trainY,pred_trainY,average = 'weighted')))
print("SVC_test  accuracy rate: {} F1-score: {}".format(svc.score(testX,testY),f1_score(testY,pred_testY,average = 'weighted')))
print("MSE:{}".format(np.mean(pred_testY-testY)**2))

SVC_train accuracy rate: 0.7116788321167883 F1-score: 0.6939000978261981
SVC_test  accuracy rate: 0.5901639344262295 F1-score: 0.56704404819422
MSE:0.04311863597002001


In [0]:
svc = svm.SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='linear',
    max_iter=5000, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state=1)

scores_accuracy = cross_val_score(svc, X, Y, cv=cv)
scores_f1 = cross_val_score(svc,X,Y,cv=cv,scoring='f1_weighted')
print(scores_accuracy.mean())
print(scores_accuracy.std())  
print(scores_f1.mean())
print(scores_f1.std())

In [0]:
svc = svm.SVC(max_iter = 5000)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state=0)
parameters = {'kernel':('linear', 'rbf','poly'),'degree':[3,5,7], 'C':[0.001, 0.01, 0.1, 1, 10],'gamma' : [ 0.01, 0.1, 1],'class_weight':['balanced',None]}
clf = GridSearchCV(svc, parameters,cv=cv)
clf.fit(X,Y)

In [0]:
clf.best_estimator_

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='linear',
    max_iter=5000, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
lr = LinearRegression()
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state=0)

scores_accuracy = cross_val_score(lr, X, Y, cv=cv,scoring='neg_mean_squared_error')
print(scores_accuracy.mean())
print(scores_accuracy.std())

-8.12717281170104
12.525109021887094


In [0]:
lasso = Lasso(alpha=0.01,max_iter=5000)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state=1)

scores_accuracy = cross_val_score(lasso, X, Y, cv=cv,scoring='neg_mean_squared_error')

print(scores_accuracy.mean())
print(scores_accuracy.std())
#sum((lasso.predict(testX)-testY)**2)/len(testX)

-0.48764704583036156
0.055006082766545845


In [0]:
ridge=Ridge(alpha=10)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state=1)

scores_accuracy = cross_val_score(ridge, X, Y, cv=cv,scoring='neg_mean_squared_error')

print(scores_accuracy.mean())
print(scores_accuracy.std())
#sum((ridge.predict(testX)-testY)**2)/len(testX)

-4.226428113241333
4.456016272069426


In [0]:
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state=0)

In [0]:
for i in cv.split(X,Y):
  print(i)

In [0]:
# lsvc = svm.LinearSVC(max_iter=100000,class_weight='balanced')
# lsvc.fit(trainX,trainY)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight='balanced', dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=100000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [0]:
pred_trainY = lsvc.predict(trainX)
pred_testY = lsvc.predict(testX)
print("SVC_train accuracy rate: {} F1-score: {}".format(lsvc.score(trainX,trainY),f1_score(trainY,pred_trainY,average = 'weighted')))
print("SVC_test  accuracy rate: {} F1-score: {}".format(lsvc.score(testX,testY),f1_score(testY,pred_testY,average = 'weighted')))

print(metrics.confusion_matrix(trainY, pred_trainY))
print(metrics.classification_report(trainY,pred_trainY, digits=3))
print(metrics.confusion_matrix(testY, pred_testY))
print(metrics.classification_report(testY,pred_testY, digits=3))

In [0]:
svc = svm.SVC(class_weight='balanced',kernel = 'linear',degree = 7,)
svc.fit(trainX,trainY)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
pred_trainY = svc.predict(trainX)
pred_testY = svc.predict(testX)
print("SVC_train accuracy rate: {} F1-score: {}".format(svc.score(trainX,trainY),f1_score(trainY,pred_trainY,average = 'weighted')))
print("SVC_test  accuracy rate: {} F1-score: {}".format(svc.score(testX,testY),f1_score(testY,pred_testY,average = 'weighted')))

print(metrics.confusion_matrix(trainY, pred_trainY))
print(metrics.classification_report(trainY,pred_trainY, digits=3))
print(metrics.confusion_matrix(testY, pred_testY))
print(metrics.classification_report(testY,pred_testY, digits=3))

In [0]:
svc = svm.SVC()
parameters = {'kernel':('linear', 'rbf','poly'),'degree':[3,5,7], 'C':[0.001, 0.01, 0.1, 1, 10],'gamma' : [ 0.01, 0.1, 1],'class_weight':['balanced',None]}
clf = RandomizedSearchCV(svc, parameters)
clf.fit(trainX,trainY)

In [0]:
clf.best_estimator_

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=7, gamma=0.01, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
clf.score(testX,testY)

0.6284153005464481

In [0]:
sum((clf.predict(testX)-testY)**2)/len(testX)

0.67003367003367

In [0]:
sorted(metrics.SCORERS.keys())

In [0]:
lasso = Lasso(max_iter=5000)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state=0)
parameters = {'alpha':[0.005, 0.01, 0.1, 1, 10]}
clf_lasso = GridSearchCV(lasso, parameters,cv=cv)
clf_lasso.fit(X,Y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7089621849428909, tolerance: 0.09029306569343058
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.11751457510519003, tolerance: 0.09029306569343057
  positive)


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=5, random_state=0, test_size=0.25,
            train_size=None),
             error_score=nan,
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=5000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [0.005, 0.01, 0.1, 1, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [0]:
clf_lasso.best_estimator_.coef_

array([-3.41058708e-03, -1.55658719e-03, -0.00000000e+00,  1.25909090e-02,
        0.00000000e+00, -2.34110232e-03,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  1.00021603e-02,  0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -6.40776608e-04,
       -3.99724191e-03, -1.63813342e-03, -0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  6.53311924e-03,  1.09091198e-02, -9.29007200e-03,
       -2.29825102e-03, -0.00000000e+00,  3.41613915e-03,  9.05376452e-03,
        0.00000000e+00,  0.00000000e+00,  1.61840684e-03, -0.00000000e+00,
       -0.00000000e+00,  6.78801516e-03,  0.00000000e+00, -3.62842844e-03,
       -0.00000000e+00, -0.00000000e+00,  1.82674369e-03, -2.57971258e-03,
        7.11745827e-03, -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -1.64205145e-02, -3.67274724e-02, -0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00,  

In [0]:
ridge = Ridge(max_iter=5000)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state=0)
parameters = {'alpha':[0.005, 0.01, 0.1, 1, 10]}
clf_ridge = GridSearchCV(ridge, parameters,cv=cv)
clf_ridge.fit(X,Y)

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=5, random_state=0, test_size=0.25,
            train_size=None),
             error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=5000, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [0.005, 0.01, 0.1, 1, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [0]:
clf_lasso.best_estimator_

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=5000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [0]:
clf_ridge.best_estimator_

Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=5000, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [0]:
ridge = Ridge()
parameters = {'alpha':[0.00, 0.01, 0.1, 1, 10]}
clf_lasso = RandomizedSearchCV(lasso, parameters)
clf.fit(trainX,trainY)

In [0]:
ridge = Ridge()
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state=0)

scores_accuracy = cross_val_score(ridge, X, Y, cv=cv,scoring='neg_mean_squared_error')

print(scores_accuracy.mean())
print(scores_accuracy.std())

-0.7250338170489272
0.5200963153279429


In [0]:
scores_accuracy

array([-4.65771695e-01, -1.79745292e+03, -2.27996813e+02, -2.78883592e+03,
       -4.75849508e-01])

In [0]:
lr = LinearRegression()
lr.fit(trainX,trainY)

In [0]:
sum((lr.predict(testX)-testY)**2)/len(testX)

In [0]:
lasso = Lasso(alpha = 0.01)
lasso.fit(trainX,trainY)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [0]:
lasso.coef_

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.02011554e-05,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [0]:
sum((clf_lasso.predict(testX)-testY)**2)/len(testX)

0.5229014401921984